In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request


In [2]:
searchQuery = "서울대입구역 양식"
searchOption = 'sim'  #sim or date  
page = 1
#python string template은 fstring으로 사용하자(문자열 앞에 f)
url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'

html = requests.get(url)



In [3]:
# html display, but not good using with theme(dark)
#from IPython.core.display import display, HTML
#display(HTML(html.text))

# parsing html 
soup = BeautifulSoup(html.text, 'html.parser')
# res = soup.find_all('li',{'class' : 'sh_blog_top'})
# for link in soup.find_all('a'):
# find_all(name, attrs, recursive, string, limit, **kwargs)
# find(name, attrs, recursive, string, **kwargs)
url_list = [];
for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
    child_item = li_item.find('a',{'class' : 'sh_blog_title'})
    title = child_item.attrs['title']
    href = child_item.attrs['href']

    #url list 저장
    url_list.append(href)
    
    print(f'title : {title} , href : {href}')
    print('----------------------------------')

    

title : [서울대입구역 맛집] 스테이크 코스가 맛있는 양식집 "모힝" , href : https://blog.naver.com/limsh0306?Redirect=Log&logNo=221878121450
----------------------------------
title : 서울대입구역, 샤로수길 맛집 ; 씨앗 양식 , href : https://blog.naver.com/hyem1211?Redirect=Log&logNo=221795755284
----------------------------------
title : [서울대입구역 / 샤로수길 저녁] 씨앗양식 - 매콤 짭조름 퓨전음식! - 특별하지는않지만, 자극적인 맛이 중독성있는곳 , href : https://jiyoung0309.blog.me/221520894644
----------------------------------
title : 샤로수길 데이트코스에는 씨앗양식이 꼭 들어가야죠! 서울대입구역 점심 저녁에도 좋아요 , href : https://lovebookn.blog.me/221627416252
----------------------------------
title : 서울여행 ① , 여행은 역시 먹방! / 서울대입구역맛집 /양식 아열대, 분식 참새방앗간, 카페 미드레벨 , href : https://blog.naver.com/haqurl1?Redirect=Log&logNo=221685341813
----------------------------------
title : 서울대입구역 샤로수길 양식 맛집 기다란 , href : https://blog.naver.com/nws4249?Redirect=Log&logNo=221809888997
----------------------------------
title : 서울대입구역 맛집 양식,한식,중식이 한자리에~ , href : https://blog.naver.com/yuneunji09?Redirect=Log&logNo=221314

In [6]:
import os
import urllib.parse

for url_item in url_list:
    nickName = "";
    postNumber = "";
    
    # url parsing , todo : blog.me 예외처리 추가해야함
    if(url_item.find("blog.me") > 0):
        parsing = url_item;
        parsing = parsing.replace("https://","")
        nickName = parsing.split('.')[0]
        postNumber = parsing.split('/')[1]
        blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber
        
    else:
        blogUrl = url_item.replace("https://", "https://m.");
    
    #https://m.blog.naver.com/soeyoung1004?Redirect=Log&logNo=221944814459

    nickName = blogUrl.split('/')[-1].split('?')[0]
    postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]
    
    if nickName is "":
        nickName = "unknown"
    if postNumber is "":
        postNumber = "1"
        
    print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
    print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
    print("blogUrl : " , blogUrl)

#     if os.path.isdir(f'image/{nickName}/{postNumber}'):
#         print("이미 수집한 경우에는 스킵")
#         continue
        
    print("------ digging more -------")
    #전체 목록 순회
    blog_html = requests.get(blogUrl)
    blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
    blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});
    index = 0;
    for div_obj in blog_image_class:
        for idx, img_item in enumerate(div_obj.find_all('img')):
            size = div_obj.find_all('img').length();

        #     print("img_item.attrs['alt'] : " , img_item.attrs['alt']);
        #     if(img_item.attrs['alt'].find("프로필") > 0):
        #         print("this is profile")
        #         continue

            img_url = img_item.attrs['src'];

            if(img_url.find("w80_blur") > 0):
                img_url = img_url.replace("w80_blur" , "w800")

            if not os.path.isdir(f'image/{nickName}/{postNumber}'):
                try:
                    os.makedirs(f'image/{nickName}/{postNumber}')

                except Exception as ex:
                    print("error catch : " , ex)
            try:
                filename = img_url.split('/')[-1].split('?')[0];
                #print('filename ' , filename);
#                 print('origin img_url : ' , img_url);
                img_url2 = urllib.parse.quote_plus(str(filename))
                #print('img_url2 : ', img_url2);
                img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#                 img_url = 'https://'+img_url2;
#                 img_url = urllib.parse.quote_plus(str(img_url)) 
#                 print(f'final img_url : {img_url}'); 
#                 img_url = urllib.parse.quote_plus(img_url);
                urllib.request.urlretrieve(img_url_final, f'image/{nickName}/{postNumber}/{index}.jpg')
                index = index +1;
            except Exception as ex:
                print("img save error : " , ex)
#                 print(f'{img_url}')


nickName :  limsh0306
postNumber :  221878121450
blogUrl :  https://m.blog.naver.com/limsh0306?Redirect=Log&logNo=221878121450
------ digging more -------


AttributeError: ResultSet object has no attribute 'length'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [5]:
print('Beginning file download with urllib2...')
#이미지다운로드 테스트용
url = 'https://m.blog.naver.com/kangmj1992?Redirect=Log&logNo=221992971007'
blog_html = requests.get(url)
blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});
index = 0;
for div_obj in blog_image_class:
    for idx,img_item in enumerate(div_obj.find_all('img')):
    #     print("img_item.attrs['alt'] : " , img_item.attrs['alt']);
    #     if(img_item.attrs['alt'].find("프로필") > 0):
    #         print("this is profile")
    #         continue

        img_url = img_item.attrs['src'];

        if(img_url.find("w80_blur") > 0):
            img_url = img_url.replace("w80_blur" , "w800")
#             print(img_url)

        if not os.path.isdir(f'image/{nickName}_test/{postNumber}'):
            try:
                os.makedirs(f'image/{nickName}_test/{postNumber}')
#                os.mkdir(f'image/{nickName}_test/{postNumber}')
#                 Exception as ex: 
            except Exception as ex:
                print("error catch : " , ex)
            #todo 한글 파일명 이슈 처리
        try:
            urllib.request.urlretrieve(img_url, f'image/{nickName}_test/{postNumber}/{index}.jpg')
            print(f'saved image path : image/{nickName}_test/{postNumber}/{index}.jpg')
            index = index + 1;
        except Exception as ex:
            print("img save error : " , ex)
            print(f'image/{nickName}/{postNumber}/{index}.jpg')

Beginning file download with urllib2...
saved image path : image/judy4883_test/221541523539/0.jpg
saved image path : image/judy4883_test/221541523539/1.jpg
saved image path : image/judy4883_test/221541523539/2.jpg
saved image path : image/judy4883_test/221541523539/3.jpg
saved image path : image/judy4883_test/221541523539/4.jpg
saved image path : image/judy4883_test/221541523539/5.jpg
saved image path : image/judy4883_test/221541523539/6.jpg
saved image path : image/judy4883_test/221541523539/7.jpg
saved image path : image/judy4883_test/221541523539/8.jpg
saved image path : image/judy4883_test/221541523539/9.jpg
saved image path : image/judy4883_test/221541523539/10.jpg
saved image path : image/judy4883_test/221541523539/11.jpg
saved image path : image/judy4883_test/221541523539/12.jpg
saved image path : image/judy4883_test/221541523539/13.jpg
saved image path : image/judy4883_test/221541523539/14.jpg
saved image path : image/judy4883_test/221541523539/15.jpg
saved image path : image/j